# Python

![](https://leukipp.github.io/xmastree/kernelspecs/python.png)

In [ ]:
import os
import js
import csv
import urllib
import numpy as np

from pyodide_js import FS
from IPython.display import HTML, Javascript, display


## Utils

In [ ]:
class FileSystemUtils(object):
    def __init__(self, fs, path):
        """ indexedDB filesystem storage """
        self.fs = fs
        self.path = path
        self.type = self.fs.filesystems.IDBFS

        self.mount()
        self.upsync()

    def mount(self):
        """ mount kernel filesystem """
        if not os.path.exists(self.path):
            self.fs.mkdir(self.path)
            self.fs.mount(self.type, {}, self.path)

    def upsync(self):
        """ sync from browser to kernel filesystem """
        self.fs.syncfs(True, lambda x: print(x) if x else None)

    def downsync(self):
        """ sync from kernel to browser filesystem """
        self.fs.syncfs(False, lambda x: print(x) if x else None)


class Simulator(object):
    def __init__(self, width='100%', height='600px'):
        """ three js simulator """
        self.width = width
        self.height = height
        self.settings = dict()
        self.rendered = False

        self.id = 'simulator'
        self.root = '' if '127.0.0.1' in str(js.location) else '/xmastree'
        self.url = f'{self.root}/files/simulator/index.html'

    def src(self):
        """ source url for iframe """
        return f'{self.url}#{urllib.parse.urlencode(self.settings)}'

    def render(self, **kwargs):
        """ render simulator html """
        self.rendered = False
        self.update(**kwargs)
        self.rendered = True
        layout = {
            'id': self.id,
            'frameborder': '0',
            'width': self.width,
            'height': self.height,
            'src': self.src()
        }
        attributes = ' '.join([f'{k}="{v}"' for k, v in layout.items()])
        display(HTML(f'<iframe allowfullscreen {attributes} />'))

    def update(self, **kwargs):
        """ update simulator settings """
        for k, v in kwargs.items():
            self.settings[k] = str(v)
        self.settings['seed'] = int(np.random.rand() * 1e6)
        if self.rendered:
            display(Javascript(f'document.getElementById("{self.id}").src="{self.src()}"'))


class Frames(list):
    def __init__(self, coords_csv, fps=30):
        """ animation frames helper """
        coords = list(csv.reader(coords_csv.split('\n')))
        self.coords = np.array([[float(col) for col in row] for row in coords])
        self.size = self.coords.shape[0]
        self.fps = fps

    def export(self, name, fs):
        """ export frames as csv """
        head = ['FRAME_ID'] + np.hstack([[f'R_{i}', f'G_{i}', f'B_{i}'] for i in range(self.size)]).tolist()
        rows = np.array(self, dtype=int)

        ids = np.arange(rows.shape[0]).reshape(-1, 1)
        frames = np.reshape(rows, (*rows.shape[:1], -1))
        data = np.hstack([ids, frames])

        path = os.path.join(fs.path, name)
        np.savetxt(path, data, header=','.join(head), fmt='%i', delimiter=',', comments='')
        fs.downsync()

        return path


## Setup

In [ ]:
# init filesystem
fs = FileSystemUtils(FS, path=os.path.join(os.path.expanduser('~'), 'data'))

# init simulator
simulator = Simulator()

# init frames
response = await js.fetch('https://raw.githubusercontent.com/standupmaths/xmastree2021/main/coords_2021.csv')
coords = await response.text()
frames = Frames(coords, fps=30)


## Code

### Carousel

- `Name of the musement ride`: It's a mess.
- `Direction of rotation`: It's a mess.

[wikipedia.org/wiki/Carousel](https://en.wikipedia.org/wiki/Carousel)


In [ ]:
# clear frames
frames.clear()


# ------------------------------------------ START CODING ------------------------------------------


def sections(coords, time, speed, division):
    """ helper function to obtain sections of an unit circle """

    # [A] obtain coordinates (only x, y is used here)
    x, y, _ = np.split(coords, 3, axis=1)

    # [B] calculate angle Θ for each coordinates (0° to 360°)
    θ = (np.arctan2(x, y) + τ) % τ

    # [C] increase angle Θ to obtain φ (counter clockwise rotation)
    φ = ((θ + np.deg2rad(time * speed)) + τ) % τ

    # [D] assign each angle φ a quadrant or octant (0 to 3 or 0 to 7)
    bins = np.array([0.0, τ/8, τ/4, 3*τ/8, τ/2, 5*τ/8, 3*τ/4, 7*τ/8, τ])[::int(8 * division)]
    idxs = np.digitize(φ, bins) - 1

    return idxs


def colors(idxs, rgb):
    """ helper function to map indices to rgb colors """
    cmap = np.vectorize(lambda i: np.array(rgb[i], dtype=object))
    return np.array([x.tolist() for x in cmap(idxs)])


# define some constant (no one likes π)
τ = 2 * np.pi

# define the run time (24 seconds)
time = np.arange(24 * frames.fps)

# define the rotation speed (direction by sign)
speed = -2.0

# write frames
for t in time:

    # [1] obtain coordinates (used for masking values)
    x, y, z = np.split(frames.coords, 3, axis=1)

    # [2] assign each section a color (mask values in center)
    carousel_idxs = sections(frames.coords, t, speed, division=1/4)
    carousel_colors = colors(carousel_idxs, [[254, 0, 0], [11, 255, 1], [1, 30, 254], [253, 254, 2]])
    carousel_colors[np.linalg.norm([x, y], axis=0) < 0.25] = [0, 0, 0]

    # [3] assign each galloper a color (mask values in center)
    galloper_idxs = sections(frames.coords, t, -1.0 * speed, division=1/8)
    galloper_colors = colors(galloper_idxs, [[237, 242, 244], [0, 0, 0], [0, 0, 0], [0, 0, 0]] * 2)
    galloper_colors[np.linalg.norm([x, y], axis=0) < 0.25] = [0, 0, 0]

    # [4] move galloper up and down (mask values above and below)
    galloper_height = 1 / 2
    galloper_center = (z.max() - z.min()) / 2.0
    galloper_offset = 0.25 * np.sin(np.deg2rad(t * speed / 2.0)) - 0.5
    galloper_top = galloper_center + galloper_offset + galloper_height
    galloper_bottom = galloper_center + galloper_offset - galloper_height
    galloper_colors[(z > galloper_top) | (z < galloper_bottom)] = [0, 0, 0]

    # [5] merge galloper colors with carousel colors (overwrite with nonzero values)
    frame = np.where(galloper_colors == 0, carousel_colors, galloper_colors)

    # append frame
    frames.append(frame)


# ------------------------------------------- END CODING -------------------------------------------


# export frames
path = frames.export('carousel_gallopers.csv', fs)

# update simulator
simulator.update(fps=frames.fps, animations=f'fs:/{path}')


In [ ]:
# render simulator
simulator.render(loop='true', rotation=0)
